In [1]:
import common

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import random

random.seed(123)
np.random.seed(123)

In [4]:
data = pd.read_csv(common.data_dir / 'Wave soldering stacked.csv')
data.head()

,A,B,C,D,E,F,G,Label,Data
0,-1,-1,-1,-1,-1,-1,-1,SqrtY1,3.605551
1,-1,-1,-1,-1,-1,-1,-1,SqrtY2,5.477226
2,-1,-1,-1,-1,-1,-1,-1,SqrtY3,5.099020
3,-1,-1,-1,1,1,1,1,SqrtY1,2.000000
4,-1,-1,-1,1,1,1,1,SqrtY2,4.000000


In [22]:
formula_main = 'Data ~ A + B + C + D + E + F + G'
formula_interaction = ('Data ~ A + B + C + G + A:B + A:C + A:G + B:C + B:G + C:G  + ' +
                       'A:B:C + A:B:G + A:C:G + B:C:G + A:B:C:G')
formula_main = formula_interaction
group = ['A', 'B', 'C', 'D', 'E', 'F', 'G']

In [21]:
resampled = data.groupby(group).sample(frac=1, replace=True).reset_index()
print(resampled)
common.buildModel(resampled, formula_main).summary().tables[1]

    index  A  B  C  D  E  F  G   Label       Data
0       1 -1 -1 -1 -1 -1 -1 -1  SqrtY2   5.477226
1       0 -1 -1 -1 -1 -1 -1 -1  SqrtY1   3.605551
2       0 -1 -1 -1 -1 -1 -1 -1  SqrtY1   3.605551
3       4 -1 -1 -1  1  1  1  1  SqrtY2   4.000000
4       4 -1 -1 -1  1  1  1  1  SqrtY2   4.000000
5       5 -1 -1 -1  1  1  1  1  SqrtY3   3.316625
6       8 -1 -1  1 -1 -1  1  1  SqrtY3   4.472136
7       6 -1 -1  1 -1 -1  1  1  SqrtY1   4.472136
8       7 -1 -1  1 -1 -1  1  1  SqrtY2   3.872983
9       9 -1 -1  1  1  1 -1 -1  SqrtY1   6.480741
10     11 -1 -1  1  1  1 -1 -1  SqrtY3   8.000000
11     10 -1 -1  1  1  1 -1 -1  SqrtY2   6.557439
12     13 -1  1 -1 -1  1 -1  1  SqrtY2   3.872983
13     14 -1  1 -1 -1  1 -1  1  SqrtY3   4.123106
14     14 -1  1 -1 -1  1 -1  1  SqrtY3   4.123106
15     15 -1  1 -1  1 -1  1 -1  SqrtY1   3.162278
16     16 -1  1 -1  1 -1  1 -1  SqrtY2   4.123106
17     16 -1  1 -1  1 -1  1 -1  SqrtY2   4.123106
18     19 -1  1  1 -1  1  1 -1  SqrtY2   5.385165


,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,5.0367,0.309,16.296,0.000,4.412,5.661
A,0.5203,0.309,1.683,0.100,-0.104,1.145
B,-0.6977,0.309,-2.257,0.030,-1.322,-0.073
C,1.4915,0.309,4.826,0.000,0.867,2.116
D,-0.1058,0.309,-0.342,0.734,-0.730,0.519
E,-0.2520,0.309,-0.815,0.420,-0.877,0.373
F,0.0323,0.309,0.104,0.917,-0.592,0.657
G,-0.7789,0.309,-2.520,0.016,-1.404,-0.154


In [25]:
ols_main = common.buildModel(data, formula_main)
ols_main.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,5.0520,0.239,21.176,0.000,4.566,5.538
A,0.5404,0.239,2.265,0.030,0.054,1.026
B,-0.5393,0.239,-2.261,0.031,-1.025,-0.053
C,1.4238,0.239,5.968,0.000,0.938,1.910
G,-0.8932,0.239,-3.744,0.001,-1.379,-0.407
A:B,-0.2714,0.239,-1.138,0.264,-0.757,0.215
A:C,0.7890,0.239,3.307,0.002,0.303,1.275
A:G,0.0261,0.239,0.109,0.914,-0.460,0.512
B:C,-0.5103,0.239,-2.139,0.040,-0.996,-0.024
B:G,0.0149,0.239,0.062,0.951,-0.471,0.501


In [23]:
ba_coeffs = common.bootstrapAnalysis(data, formula_main, nrepeats=1_000)
bba_coeffs = common.befittingBootstrapAnalysis(data, formula_main, group, nrepeats=1_000)

In [9]:
#ba_coeffs.agg(['mean', 'std']).transpose()

In [10]:
# bba_coeffs.agg(['mean', 'std']).transpose()

In [24]:
# Table IV 
tab_4 = pd.DataFrame({
    'BBA mean': bba_coeffs.mean(),
    'BBA var': bba_coeffs.var(),
    'BBA se': bba_coeffs.std(),
    'BBA LCI': bba_coeffs.quantile(0.025, axis=0),
    'BBA UCI': bba_coeffs.quantile(0.975, axis=0),
    
})
tab_4['95% BCI'] = tab_4['BBA UCI'] - tab_4['BBA LCI']
tab_4['to SE'] = tab_4['95% BCI'] / 3.92
tab_4.round(4)
# bba_coeffs.quantile(0.975, axis=0) - 

,BBA mean,BBA var,BBA se,BBA LCI,BBA UCI,95% BCI,to SE
Intercept,5.0578,0.0380,0.1948,4.6887,5.4359,0.7472,0.1906
A,0.5410,0.0385,0.1961,0.1620,0.9329,0.7709,0.1967
B,-0.5398,0.0381,0.1951,-0.9414,-0.1648,0.7767,0.1981
C,1.4290,0.0386,0.1964,1.0714,1.8192,0.7477,0.1907
G,-0.8918,0.0396,0.1991,-1.2628,-0.4976,0.7652,0.1952
A:B,-0.2704,0.0380,0.1949,-0.6568,0.0883,0.7451,0.1901
A:C,0.7930,0.0397,0.1993,0.4225,1.2084,0.7859,0.2005
A:G,0.0308,0.0387,0.1967,-0.3291,0.4127,0.7418,0.1892
B:C,-0.5072,0.0391,0.1978,-0.9038,-0.1326,0.7712,0.1967
B:G,0.0120,0.0397,0.1992,-0.4023,0.3801,0.7824,0.1996


In [12]:
tab = pd.DataFrame({
    'BA mean': ba_coeffs.mean(),
    'BA var': ba_coeffs.var(),
    'BA se': ba_coeffs.std(),
    'BA LCI': ba_coeffs.quantile(0.025, axis=0),
    'BA UCI': ba_coeffs.quantile(0.975, axis=0),
    
})
tab['95% BCI'] = tab['BA UCI'] - tab['BA LCI']
tab['to SE'] = tab['95% BCI'] / 3.92
tab.round(4)
# bba_coeffs.quantile(0.975, axis=0) - 

,BA mean,BA var,BA se,BA LCI,BA UCI,95% BCI,to SE
Intercept,5.0457,0.1083,0.3291,4.4139,5.7346,1.3207,0.3369
A,0.5216,0.0950,0.3083,-0.0756,1.1714,1.2470,0.3181
B,-0.5291,0.0979,0.3129,-1.1541,0.0677,1.2218,0.3117
C,1.4462,0.0995,0.3154,0.8481,2.0947,1.2466,0.3180
D,-0.1307,0.0974,0.3121,-0.7099,0.4662,1.1761,0.3000
E,-0.3041,0.1008,0.3174,-0.9101,0.3367,1.2467,0.3180
F,-0.1661,0.0998,0.3160,-0.8422,0.4208,1.2629,0.3222
G,-0.8884,0.0967,0.3110,-1.4574,-0.2322,1.2252,0.3125
